### **[LeetCode Link](https://leetcode-cn.com/problems/n-queens/solution/gen-ju-di-46-ti-quan-pai-lie-de-hui-su-suan-fa-si-/)**

## 回溯搜索算法（深度优先遍历）
### 解题思路：
一句话题解：回溯算法是一种遍历算法，以 **深度优先遍历** 的方式尝试所有的可能性。回溯算法是 **有方向地** 搜索，区别于多层循环实现的暴力法。
### 考虑对角线（找规律）
![](1.png)
为了保证至少两个皇后不同时出现在 **同一主对角线方向** 或者 **同一副对角线方向**。检查策略是，只要「检测」到新摆放的「皇后」与已经摆放好的「皇后」冲突，就尝试摆放同一行的下一个位置，到行尾还不能放置皇后，**就退回到上一行。**
### 复杂度分析
* 时间复杂度：$O(N!)$，其中 $N$ 是皇后数量。
* 空间复杂度：$O(N)$，其中 $N$ 是皇后数量。空间复杂度主要取决于递归调用层数、记录每行放置的皇后的列下标的数组以及三个集合，递归调用层数不会超过 $N$，数组的长度为 NN，每个集合的元素个数都不会超过 $N$。

In [ ]:
class Solution:
    def solveNQueens(self, n: int) -> List[List[str]]:
        diagonal = collections.defaultdict(list)
        res = []
        column = set()
        main_diagonal = set()
        sub_diagonal = set()

        def backtrack(x, y, count):
            if x in diagonal['row'] or y in diagonal['column'] or visited[x][y] == 'Q':
                return 
            visited[x] = visited[x][:y] + 'Q' + visited[x][y+1:]
            column.add(y)
            main_diagonal.add(x - y)
            sub_diagonal.add(x + y)
            if count == n-1:
                res.append(visited[:])
            for i in range(n):
                if i in column or x + 1 - i in main_diagonal or x + 1 + i in sub_diagonal:
                    continue
                backtrack(x + 1, i, count + 1)
            visited[x] = visited[x][:y] + '.' + visited[x][y+1:]
            column.remove(y)
            main_diagonal.remove(x - y)
            sub_diagonal.remove(x + y)

        for i in range(n):
            visited = ['.' * n] * n
            backtrack(0, i, 0)
        return res

### **[LeetCode Link](https://leetcode-cn.com/problems/n-queens-ii/solution/dfs-wei-yun-suan-jian-zhi-by-makeex/275321)**

## 使用 bitmap 回溯
### 核心思路：
* 使用常规深度优先一层层搜索
* 使用三个整形分别标记每一层哪些格子可以放置皇后，这三个整形分别代表列、左斜下、右斜下`(_col, ld, rd_)`，二进制位为 1 代表不能放置，0 代表可以放置
* 核心两个位运算：
 1. x & -x 代表除最后一位 1 保留，其它位全部为 0
 2. x & (x - 1) 代表将最后一位 1 变成 0
 
### 复杂度分析
* 时间复杂度：$\mathcal{O}(N!)$.
* 空间复杂度：$\mathcal{O}(N)$.

In [70]:
class Solution:
    def totalNQueens(self, n: int) -> int:
        """
        :type n: int
        :rtype: int
        """
        def backtrack(row = 0, hills = 0, next_row = 0, dales = 0, count = 0):
            """
            :type row: 当前放置皇后的行号
            :type hills: 主对角线占据情况 [1 = 被占据，0 = 未被占据]
            :type next_row: 下一行被占据的情况 [1 = 被占据，0 = 未被占据]
            :type dales: 次对角线占据情况 [1 = 被占据，0 = 未被占据]
            :rtype: 所有可行解的个数
            """
            if row == n:  # 如果已经放置了 n 个皇后
                count += 1  # 累加可行解
            else:
                # 当前行可用的列
                # ! 表示 0 和 1 的含义对于变量 hills, next_row and dales的含义是相反的
                # [1 = 未被占据，0 = 被占据]
                free_columns = columns & ~(hills | next_row | dales)
                # 找到可以放置下一个皇后的列
                while free_columns:
                    # free_columns 的第一个为 '1' 的位
                    # 在该列我们放置当前皇后
                    curr_column = - free_columns & free_columns

                    # 放置皇后
                    # 并且排除对应的列
                    free_columns ^= curr_column

                    count = backtrack(row + 1, 
                                    (hills | curr_column) << 1, 
                                    next_row | curr_column, 
                                    (dales | curr_column) >> 1, 
                                    count)
            return count

        # 棋盘所有的列都可放置，
        # 即，按位表示为 n 个 '1'
        # bin(cols) = 0b1111 (n = 4), bin(cols) = 0b111 (n = 3)
        # [1 = 可放置]
        columns = (1 << n) - 1
        return backtrack()